In [ ]:
import sys
sys.path.append("./drive_interfaces/carla/carla_client/")
from carla.client import make_carla_client
from carla.settings import CarlaSettings
from carla.client import VehicleControl

In [ ]:
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    print(f.read())

In [ ]:
import random
from carla.sensor import Camera, Lidar
settings = CarlaSettings()
settings.set(
    SynchronousMode=True,
    SendNonPlayerAgentsInfo=True,
    NumberOfVehicles=20,
    NumberOfPedestrians=40,
    WeatherId=random.choice([1, 3, 7, 8, 14]),
    QualityLevel="Low")
settings.randomize_seeds()

# Now we want to add a couple of cameras to the player vehicle.
# We will collect the images produced by these cameras every
# frame.

# The default camera captures RGB images of the scene.
camera0 = Camera('CameraRGB')
# Set image resolution in pixels.
camera0.set_image_size(800, 600)
# Set its position relative to the car in meters.
camera0.set_position(0.30, 0, 1.30)
settings.add_sensor(camera0)

# Let's add another camera producing ground-truth depth.
camera1 = Camera('CameraDepth', PostProcessing='Depth')
camera1.set_image_size(800, 600)
camera1.set_position(0.30, 0, 1.30)
settings.add_sensor(camera1)

print(str(settings))

In [ ]:
from carla.client import CarlaClient
client = CarlaClient("127.0.0.1", 2000)
client.connect()
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    #client.load_settings(settings)
    client.load_settings(f.read())
client.start_episode(0)
measurements, sensor_data = client.read_data()
client.send_control(VehicleControl())

client.disconnect()

In [ ]:
with make_carla_client("127.0.0.1", 2000) as client:
    with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
        #client.load_settings(settings)
        client.load_settings(f.read())
    client.start_episode(0)
    measurements, sensor_data = client.read_data()
    client.send_control(VehicleControl())

In [ ]:
measurements.player_measurements

In [ ]:
sensor_data.keys()

In [ ]:
image_converter.depth_to_array(sensor_data['DepthLeft']).shape

In [ ]:
a=image_converter.labels_to_array(sensor_data['SegLeft'])

In [ ]:
import numpy as np
b=a[:,:,np.newaxis]

In [ ]:
b.shape

In [ ]:
import scipy.misc
image = scipy.misc.imresize(b, [30, 40], interp='nearest')

In [ ]:
a=sensor_data['DepthLeft']

In [ ]:
from carla import image_converter

In [ ]:
b=image_converter.depth_to_array(a)
b.shape

In [ ]:
b

In [ ]:
measurements

In [ ]:
measure, sensor =

In [2]:
path= "/scratch/yang/aws_data/carla_collect/1/2018726_default_RotationPitch=-15_WeatherId=10_4/data_00000.h5"
import h5py
f=h5py.File(path, "r")

/home/yang/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
f.keys()

In [ ]:
import cv2
image = cv2.imdecode(f['CameraLeft'][0], 1)

In [ ]:
from PIL import Image
a=Image.fromarray(image)

In [ ]:
a

In [10]:
path= "/scratch/yang/aws_data/carla_collect/1/2018726_default_RotationPitch=-15_WeatherId=10_5/data_00000.h5"
path = "/data/yang/code/aws/scratch/carla_collect/1/2018727_default_RotationPitch=0_WeatherId=13_1/data_00000.h5"
path = "/scratch/yang/aws_data/carla_collect/1/2018730_default_RotationPitch=0_WeatherId=01_1/data_00000.h5"
def sample_images_from_h5(path):
    f=h5py.File(path, "r")
    for key in f.keys():
        if 'targets' not in key:
            for imid in range(200):
                data = f[key][imid]
                if 'camera' in key.lower():
                    ext = ".jpg"
                else:
                    ext = ".png"
                out_name = path+"." + key + str(imid).zfill(4) + ext
                with open(out_name, "w") as g:
                    g.write(data)
sample_images_from_h5(path)

"ffmpeg -i data_00000.h5.CameraMiddle%04d.png -c:v libx264 out.mp4"

'ffmpeg -i data_00000.h5.CameraMiddle%04d.png -c:v libx264 out.mp4'

In [14]:
from subprocess import call
for i in range(1):
    prefix = "/scratch/yang/aws_data/carla_collect/1/2018730_default_RotationPitch=0_WeatherId="+str(i).zfill(2)+"_1/"
    path = prefix + "data_00000.h5"
    ! ls -alh $path
    sample_images_from_h5(path)
    
    call("ffmpeg -i "+ prefix +"data_00000.h5.CameraMiddle%04d.jpg -c:v libx264 "+ prefix[:-1]+".mp4", shell=True)

-rw-r--r-- 1 yang users 48M Jul 30 23:41 /scratch/yang/aws_data/carla_collect/1/2018730_default_RotationPitch=0_WeatherId=00_1/data_00000.h5


In [1]:
import threading
from multiprocessing import Process
import time

class ControlInterface(object):
    def __init__(self):
        self._throttle = 1.0       
        self.counter = 0        
        self.start_loop()
        self.last_time = time.time()

    def set_throttle(self, new_throttle):
        self._throttle = new_throttle

    def start_loop(self):
        self.counter += 1
        if self.counter % 75 == 0:
            print("time eplapsed is ", time.time()-self.last_time)
            self.last_time = time.time()
            print( "throttle", self._throttle)
        threading.Timer(1.0/75, self.start_loop).start()
        

ctrl = ControlInterface()
print("after constructor")
ctrl.set_throttle(-1.0)

after constructor
('time eplapsed is ', 1.0166008472442627)
('throttle', -1.0)
('time eplapsed is ', 1.0324769020080566)
('throttle', -1.0)
('time eplapsed is ', 1.0310008525848389)
('throttle', -1.0)
('time eplapsed is ', 1.0606470108032227)
('throttle', -1.0)
('time eplapsed is ', 1.0337679386138916)
('throttle', -1.0)
('time eplapsed is ', 1.0316898822784424)
('throttle', -1.0)
('time eplapsed is ', 1.032865047454834)
('throttle', -1.0)
('time eplapsed is ', 1.0337390899658203)
('throttle', -1.0)


In [18]:
import os
def get_files_count_in_folder(folder):
    if not os.path.exists(folder):
        return 0
    else:
        pass
import glob
len(glob.glob("/data/yang/code/aws/scratch/carla_collect/1/2018731_default_RotationPitch=0_WeatherId=01_1/*.h5"))

27

In [44]:
import os, threading
cmd = ['bash', '-c', " '/scratch/yang/aws_data/carla_0.8.4/CarlaUE4.sh  -carla-server -carla-settings=/data/yang/code/aws/CIL_modular/drive_interfaces/carla/yang_template.ini -benchmark -fps=5' "]
#cmd = cmd.split(" ")
#cmd = [x for x in cmd if x!="" ]
print(" ".join(cmd))
print("before spawnling")
t = threading.Thread(target = lambda : os.system(" ".join(cmd)))
t.start()       
#! ps aux | grep $id0

bash -c  '/scratch/yang/aws_data/carla_0.8.4/CarlaUE4.sh  -carla-server -carla-settings=/data/yang/code/aws/CIL_modular/drive_interfaces/carla/yang_template.ini -benchmark -fps=5' 
before spawnling


In [46]:
len(glob.glob("/data/yang/code/aws/scratch/carla_collect/1/*/data_*.h5"))

4212

In [1]:
path = "/scratch/yang/aws_data/CIL_modular_data/matthias_data/RC025Val/data_00000.h5"
import h5py

/home/yang/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f=h5py.File(path, "r")

a=f['rgb']

In [ ]:
from PIL import Image
i+=3
print(i)
Image.fromarray(a[i, :,:,:])

In [3]:
path ="/scratch/yang/aws_data/carla_collect/1/default_ImageSizeX=700_WeatherId=10/data_00099.h5"
import h5py, time
f = h5py.File(path, "r")

/home/yang/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
import cv2
start = time.time()
t_decode = 0.0
t_resize = 0.0
for i in range(200):
    a= f["CameraMiddle"][i]
    start = time.time()
    decoded = cv2.imdecode(a, 1)                    
    t_decode += time.time()-start
    sz = (88,200)
    start = time.time()
    decoded = cv2.resize(decoded, (sz[1], sz[0]))
    t_resize += time.time() -start
delta = time.time()-start
print(delta, 200/delta)
print(t_decode, t_resize)

(0.00037789344787597656, 529249.716088328)
(0.9612674713134766, 0.06985902786254883)


In [99]:
91*20*200 /1000/1000.0

0.364

In [101]:
8*75/1.5

400.0

In [6]:
from joblib import Parallel, delayed
p = Parallel(n_jobs=8, backend="threading", verbose=10)


func = lambda x : cv2.imdecode(x, 1)
if True:
    height, width = 88, 200
    func_previous = func
    func = lambda x : cv2.resize(func_previous(x), (width, height))
                
func = lambda x : cv2.resize(cv2.imdecode(x, 1), (200, 88))        
func = delayed(func)
results = p(func(f["CameraMiddle"][0]) for i in range(8*75))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Para

In [7]:
results = p(delayed(cv2.imdecode)(f["CameraMiddle"][i], 1) for i in range(200))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Para

In [ ]:
from PIL import Image
Image.fromarray(results[0])

In [13]:
import numpy as np
a = np.concatenate((results[0], results[0]+5), axis=2)

In [116]:
b=cv2.resize(a, (200, 300))


In [117]:
b.shape

(300, 200, 6)

In [ ]:
Image.fromarray(b[:,:,:3])

In [30]:

def _resize_BHWC(input, sz):
    B, H, W, C = input.shape
    input = np.transpose(input, (1, 2, 3, 0))
    # now input has size H W C B
    input = np.reshape(input, (H, W, C*B))
    print(input.shape)
    output = cv2.resize(input[:,:,:1000], (sz[1], sz[0]))
    output = np.reshape(output, (sz[0], sz[1], C, B))
    output = np.transpose(output, (3, 0, 1, 2))
    return output


In [15]:
stk = np.stack(results, axis = 0)

In [16]:
stk.shape

(200, 576, 768, 3)

In [31]:
out = _resize_BHWC(stk, (200, 300))

(576, 768, 600)


error: OpenCV(3.4.1) /io/opencv/modules/imgproc/src/resize.cpp:3687: error: (-215) dsize.area() > 0 in function resize


In [ ]:
func = lambda x: cv2.imdecode(x, 1)
if hasattr(self._config, "hack_resize_image"):
    height, width = self._config.hack_resize_image
    func_previous = func
    func = lambda x: cv2.resize(func_previous(x), (width, height))
